# Comparing the performance of different clustering algorithms on toy datasets on adding high dimensional Gaussian noise

This experiment analyzes how different clustering algorithms perform on different datasets that are originally two dimensional when high dimensional Gaussian noise is added to them. The experiment also shows how the performance of the clutstering algorithms in the above simulations is affected when the variance of the Gaussian noise added is varied. This experiment is influenced by Scikitlearn's demo on clustering, 'Comparing different clustering algorithms on toy datasets'and is carried out with the aim of analyzing the behavior of different clustering algoithms when dealing with high dimensional data.


In [1]:
import time
import warnings
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
from sklearn import preprocessing

from sklearn.metrics import adjusted_rand_score

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

plt.rcParams.update({'font.size': 20})

## Generating toy datasets and defining the parameters for the clustering algorithms

The clustering algorithms whose performance is being evaluated are k-means clustering, agglomerative clustering, DBSCAN, GMM, spectral clustering, and affinity propagation. 

The 2D toy datasest are generated using make_blobs,make_classification, make_moons, and make_circles from sklearn.datasets. The generated datasets all of which have 500 samples with 2 features that belong to 2 classes are normalized before noise addition so as to make the variance 1, and centralize the data points around zero which enables easier parameter selection while clustering.

In [2]:
'''defining datasets'''
noise_dimensions=np.arange(0,110,10)
noise_levels=[0.1,1,10]
noise_range=len(noise_dimensions)
trial_num=100
parameters={'samples':500,'features':2,'clusters':2,'eps': .3,'damping':0.9,'preference':-200}

initial_blobs = datasets.make_blobs(n_samples=parameters['samples'], 
                            centers=parameters['clusters'],
                            cluster_std=1,
                            random_state=8)

initial_moons = datasets.make_moons(n_samples=parameters['samples'], 
                            noise=0.06)
initial_circles = datasets.make_circles(n_samples=parameters['samples'], 
                                factor=0.5,
                                noise=0.06)
initial_classification = datasets.make_classification(n_samples=parameters['samples'], 
                                  n_features=parameters['features'],
                                  n_informative=parameters['features'],
                                  n_redundant=0,
                                  n_repeated=0,
                                  n_classes=parameters['clusters'],
                                  n_clusters_per_class=1,
                                  flip_y=0.1,
                                  random_state=8,
                                  shuffle=True)

'''Normalizing all the datasets for easier parameter selection'''
def standardize(input_data): # input should be the dataset
    temp = preprocessing.StandardScaler().fit_transform(input_data[0])
    standardized_input = []
    standardized_input.append(temp)
    standardized_input.append(input_data[1])
    return standardized_input

blobs = standardize(initial_blobs)
classification = standardize(initial_classification)
circles = standardize(initial_circles)
moons = standardize(initial_moons)

datasets = [blobs,classification,circles,moons]


names=['make_blobs','make_classification','make_circles','make_moons']

clustering_algorithms = [cluster.KMeans(n_clusters=parameters['clusters']),
                         cluster.AgglomerativeClustering(n_clusters=parameters['clusters'],linkage='ward'),
                         cluster.DBSCAN(eps=parameters['eps']),
                         mixture.GaussianMixture(n_components=parameters['clusters'],covariance_type='full'),
                         cluster.SpectralClustering(n_clusters=parameters['clusters'],eigen_solver='arpack',affinity='nearest_neighbors'),
                         cluster.AffinityPropagation(damping=parameters['damping'],preference=parameters['preference'])]
clustering_names=['K-means Clustering','Agglomerative Clustering','DBSCAN','GMM','Spectral Clustering','Affinity Propagation']

## Adding noise and computing the accuracy
In each simulation different clustering algorithms are performed after Gaussian noise of dimension n is added to the generated datasets where n=0,10,...,100. The performance of the clustering algorithms is measured using adjusted rand index (ARI) as the metric. 

The above experiment is carried out by changing the variance of the added noise to analyze also the effect of increasing variance in the above simulations. A noise with standard deviation of 0.1 is regarded as having low variance, 1 as having moderate variance, and 10 as having high variance in this context. The Gaussian noise added is always centered around zero. There are 100 trials for this experiment, and the mean and variance resulting from these 100 trails are used in plotting the results so as to minimize random effects in the experiment.

In [3]:
'''Adding high dimensional noise'''
def addNoise(x,noise_dim,noise_std,samples=parameters['samples'],noise_mean=0):
    temp_noise=np.random.normal(noise_mean,noise_std,[samples,noise_dim])
    data=np.concatenate((x,temp_noise),axis=1)
    return data

In [4]:
def accuracy(algorithm,data_X,data_y):
    cluster=algorithm.fit(data_X)
    label=cluster.labels_
    ARI=adjusted_rand_score(label,data_y)
    return ARI

def accuracyGaussian(algorithm,data_X,data_y):
    cluster=algorithm.fit(data_X)
    label=cluster.predict(data_X)
    ARI=adjusted_rand_score(label,data_y)
    return ARI

def toPlot(noise_std):
    ARI=np.zeros((len(datasets),len(clustering_algorithms),trial_num,noise_range))
    all_trial_ARI=np.zeros((len(datasets),len(clustering_algorithms),noise_range*trial_num)) # to create dataframe
    the_noise_dim=np.zeros((len(datasets),len(clustering_algorithms),noise_range*trial_num))
    all_ARI=[[],[],[],[]]
    event=[[],[],[],[]]
    all_noise=[[],[],[],[]]
    for index,dataset in enumerate(datasets):
        for ind,algorithm in enumerate(clustering_algorithms):
            for trial in range(trial_num):
                for noise_index,noise in enumerate(noise_dimensions):
                    new_X=addNoise(dataset[0],noise,noise_std)
                    if ind==3:
                        ARI[index][ind][trial][noise_index]=accuracyGaussian(algorithm,new_X,dataset[1])
                    else:
                        ARI[index][ind][trial][noise_index]=accuracy(algorithm,new_X,dataset[1])
                    all_trial_ARI[index][ind][noise_index+trial*noise_range]=ARI[index][ind][trial][noise_index]
                    the_noise_dim[index][ind][(noise_index)+trial*noise_range]=noise
                    #to create a combined plot
                    all_ARI[index].append(ARI[index][ind][trial][noise_index])
                    event[index].append(clustering_names[ind])
                    all_noise[index].append(noise)
                    
    return all_noise,event,all_ARI

all_noise_1,event_1,all_ARI_1=toPlot(0.1) # arrays corresponding to noise_std=0.1
all_noise_2,event_2,all_ARI_2=toPlot(1) # arrays corresponding to noise_std=1
all_noise_3,event_3,all_ARI_3=toPlot(10) # arrays corresponding to noise_std=10

Plotting the number of noise dimensions against ARI enables a better visualization of change in the performance of each clustering algorithm with the number of noise dimensions for different datasets. The three plots that follow give a better intuition on how the whole scenario changes when dealing with noise of three different variances.

In [ ]:
plt.rcParams.update({'font.size': 18})
'''Plotting the data'''
f, axes = plt.subplots(len(noise_levels)+1, len(datasets), figsize=(25, 20), sharex=True,sharey=True)
for index,(dataset,name) in enumerate(zip(datasets,names)):
    plt.subplot(len(noise_levels)+1,len(datasets),index+1)
    ax=sns.scatterplot(dataset[0][:,0],dataset[0][:,1],hue=dataset[1],edgecolor='black')
    plt.title(name)
    plt.tight_layout()    
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    if(index+1!=len(datasets)):
        ax.legend_.remove()
    plt.ylim(-2.5,2.5)
    plt.xlim(-3.3,3.3)
    plt.tight_layout()


all_ARI=[all_ARI_1,all_ARI_2,all_ARI_3]
all_noise=[all_noise_1,all_noise_2,all_noise_3]
event=event_1 # event_1, event_2, and event_3 are all the same arrays
def plotAll(z,dimension,rand_index,dataset,noise_level,event=event[0],x=len(noise_levels)+1,y=len(datasets)):
    plt.subplot(x,y,z)
    data={'Number of Noise Dimensions':dimension,'ARI':rand_index,'Event':event}
    ax=sns.lineplot(x='Number of Noise Dimensions',y='ARI',hue='Event',data=data)
    if (z%len(datasets)==1):
        plt.ylabel(' noise_std='+ str(noise_level))
    else:
        plt.gca().yaxis.set_major_locator(plt.NullLocator())
    if(z<=(len(noise_levels))*len(datasets)):
        plt.gca().xaxis.set_major_locator(plt.NullLocator())
    if(z<=len(datasets)):
        plt.title(dataset)    
    plt.tight_layout()
    if(z!=len(datasets)*2):
        ax.legend_.remove()
    if(z==len(datasets)*2):
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.ylim(-0.1,1.1)
    return data

f.text(0.425, -0.012, 'Number of Noise Dimensions', ha='center',color='b',fontsize=22)
f.text(-0.022, 0.385, 'ARI', va='center', rotation='vertical',color='b',fontsize=22)
for index,noise_level in enumerate(noise_levels):
    for ind,(dataset,name) in enumerate(zip(datasets,names)):
        plotAll(index*len(datasets)+5+ind,all_noise[index][ind],all_ARI[index][ind],name,noise_level)

## Result interpretation

When the noise standard deviation is low (noise_std=0.1), spectral clustering seems to give the best performance. Spectral clustering has a near perfect ARI when applied to make_blobs and make_moons datasets and this doesn't seem to be affected on increasing the noise dimension from 0 to 100. Also, it shows better performance compared to other clustering algoithms in the case of make_classification. To add, this is the only algorithm that does a good job in clustering in the case of make_circles other than DBSCAN. Here while spectral clustering has a near-perfect ARI on the addition of upto 50 noise dimensions, the accuracy of DBSCAN drops to zero soon after noise of any dimension is added. The performances of agglomerative clustering, affinity propagation, and k-means clustering algorithms don't seem to change on increasing the noise dimensions but at the same time give out the worst performance with ARI nearly equal to zero in the case of make_circles. Although GMM is similar to these thre algorithms in performance, it is affected more by the addition of increasing noise dimensions. 


DBSCAN performs consistently poorly on applying to make_classification. In other datasets although it has a near perfect ARI when no noise dimension is added, the performance plummets down and reaches close to zero as soon as noise of any dimension is added. In summary, spectral clustering does a better job compared to other algorithms under consideration in terms of performance and susceptibility to increasing addition of noise dimensions.

When the noise standard deviation is moderate (noise_std=1), spectral clustering again performs better compared to others and is comparatively less susceptible to increasing noise dimesions. All algorithms seem to fail in clustering when noise of any dimension with high variance (noise_std=10) is added.

To summarize, spectral clustering puts forth a better performance when noise with moderate or low variance is added.